# Fraud Detection with Graph databases and Machine Learning

## Importing the required Python libraries

In [120]:
import numpy as np
import pandas as pd
from py2neo import Graph
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from imblearn.over_sampling import SMOTE 
from collections import Counter
from sklearn.decomposition import PCA

## Loading and exploring the banksim dataset 

In [134]:
banksim_df = pd.read_csv("../data/bs140513_032310.csv")
banksim_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [135]:
banksim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
step           594643 non-null int64
customer       594643 non-null object
age            594643 non-null object
gender         594643 non-null object
zipcodeOri     594643 non-null object
merchant       594643 non-null object
zipMerchant    594643 non-null object
category       594643 non-null object
amount         594643 non-null float64
fraud          594643 non-null int64
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB


Viewing the split of the output classes - fraudulent and genuine transactions

In [136]:
banksim_df['fraud'].value_counts()

0    587443
1      7200
Name: fraud, dtype: int64

In [124]:
# Number of unique values per column in the banksim dataset
banksim_df.nunique()

step             180
customer        4112
age                8
gender             4
zipcodeOri         1
merchant          50
zipMerchant        1
category          15
amount         23767
fraud              2
dtype: int64

## Preprocessing the data 

In [125]:
# Obtaining the number of null values in each column
banksim_df.isna().sum()

step           0
customer       0
age            0
gender         0
zipcodeOri     0
merchant       0
zipMerchant    0
category       0
amount         0
fraud          0
dtype: int64

Looks like there are no null values in the dataset.

In [126]:
# Retrieving the class attribute from the dataframe
Y_before_smote = banksim_df['fraud']
Y_before_smote.head()

0    0
1    0
2    0
3    0
4    0
Name: fraud, dtype: int64

In [127]:
'''
Removing unwanted columns
Since zipcodeOri and zipMerchant have the same value for all the rows, these columns are redundant
'''

feature_df = banksim_df.drop(['step', 'customer', 'zipcodeOri', 'zipMerchant', 'fraud'], axis=1)

# Encoding the categorical variables because SMOTE cannot work with string values
label_encoder = LabelEncoder()
feature_df['gender'] = label_encoder.fit_transform(feature_df['gender'])
feature_df['age'] = label_encoder.fit_transform(feature_df['age'])
feature_df['category'] = label_encoder.fit_transform(feature_df['category'])
feature_df['merchant'] = label_encoder.fit_transform(feature_df['merchant'])


In [128]:
feature_df.head()

,age,gender,merchant,category,amount
0,4,2,30,12,4.55
1,2,2,30,12,39.68
2,4,1,18,12,26.89
3,3,2,30,12,17.25
4,5,2,30,12,35.72


In [113]:
# Handling the imbalance in the dataset using SMOTE

print('Original dataset shape %s' % Counter(Y_before_smote))

X_before_smote = feature_df
sm = SMOTE()#random_state=1)

# Applying smote to dataset, result is nparray
X_after_smote, Y_after_smote = sm.fit_resample(X_before_smote, Y_before_smote)
                                               
# Converting back tp data frame from nparray
X_after_smote_df = pd.DataFrame(X_after_smote, columns=X_before_smote.columns)
Y_after_smote_df = pd.DataFrame(Y_after_smote, columns = ["fraud"])
                                               
print('dataset shape after smote %s' % Counter(Y_after_smote))

Original dataset shape Counter({0: 587443, 1: 7200})
dataset shape after smote Counter({0: 587443, 1: 587443})


In [114]:
# Verifying the number of unique values in each column after SMOTE
X_after_smote_df.nunique()

age              8
gender           4
merchant        50
category        15
amount      602548
dtype: int64

In [129]:
# Converting the encoded merchant values back to the original string values
#X_after_smote_df['merchant'] = label_encoder.inverse_transform(X_after_smote_df['merchant'])

# One hot encoding the categorical variables
feature_df = pd.get_dummies(X_after_smote_df, columns=['age', 'gender', 'category'], 'merchant'])
#feature_df = X_after_smote_df
feature_df.head()

,age,gender,merchant,category,amount
0,4,2,30,12,4.55
1,2,2,30,12,39.68
2,4,1,18,12,26.89
3,3,2,30,12,17.25
4,5,2,30,12,35.72


In [130]:
# Standardizing the features
standard_scaler = StandardScaler()
scaled_df = pd.DataFrame(standard_scaler.fit_transform(feature_df), columns = feature_df.columns)

scaled_df.head()

,age,gender,merchant,category,amount
0,0.737975,1.090643,0.714001,0.331405,-0.299276
1,-0.759162,1.090643,0.714001,0.331405,0.016067
2,0.737975,-0.896279,-0.682938,0.331405,-0.098742
3,-0.010593,1.090643,0.714001,0.331405,-0.185275
4,1.486544,1.090643,0.714001,0.331405,-0.019480


In [131]:
# Performing dimensionality reduction using PCA

# Limiting the number of components such that 95% of the variance is explained
#pca = PCA(0.95, svd_solver='full')
#scaled_df = pca.fit_transform(scaled_df)

scaled_df.shape


(594643, 5)

## Training supervised learning models using intrinsic features from the dataset

In [132]:
k_fold = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

random_forest = RandomForestClassifier(max_depth=4, n_estimators=150, verbose=True)
svm = SVC(gamma="auto")
logistic_regression = LogisticRegression(solver='lbfgs')
scaled_df = scaled_df.values
#labels = Y_after_smote
labels = Y_before_smote

In [33]:
# Random Forest Classifier

for train_index, test_index in k_fold.split(scaled_df):

    X_train, X_test = scaled_df[train_index], scaled_df[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    #X_train, X_test, y_train, y_test = train_test_split(scaled_df, labels, test_size=0.20, random_state=42)

    clf = random_forest.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions) * 100

    
'''
    metrics = precision_recall_fscore_support(y_test, predictions, average='macro')
    precision, recall = metrics[0], metrics[1]
    f1_score = 2 * (precision * recall) / (precision + recall)

    
    print("Accuracy = {0:.4f}%".format(accuracy))
    print("Precision = {0:.4f}".format(precision))
    print("Recall = {0:.4f}".format(recall))
    print("F1 Score = {0:.4f}".format(f1_score))'''
    print(classification_report(y_test, predictions))
    print("---------------------------------------------")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  9.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    2.1s finished


              precision    recall  f1-score   support

           0       1.00      0.94      0.97    117625
           1       0.94      1.00      0.97    117353

    accuracy                           0.97    234978
   macro avg       0.97      0.97      0.97    234978
weighted avg       0.97      0.97      0.97    234978



In [13]:
# SVM Classifier
'''
for train_index, test_index in k_fold.split(scaled_df):
    X_train, X_test = scaled_df[train_index], scaled_df[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = svm.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions) * 100

    metrics = precision_recall_fscore_support(y_test, predictions, average='macro')
    precision, recall = metrics[0], metrics[1]
    f1_score = 2 * (precision * recall) / (precision + recall)

    
    print("Accuracy = {0:.4f}%".format(accuracy))
    print("Precision = {0:.4f}".format(precision))
    print("Recall = {0:.4f}".format(recall))
    print("F1 Score = {0:.4f}".format(f1_score))
    print("---------------------------------------------")'''

Finished training
Accuracy = 99.3610%
Precision = 0.9014
Recall = 0.8650
F1 Score = 0.8828
---------------------------------------------
Finished training
Accuracy = 99.5434%
Precision = 0.9360
Recall = 0.8705
F1 Score = 0.9021
---------------------------------------------
Finished training
Accuracy = 99.5628%
Precision = 0.9406
Recall = 0.8588
F1 Score = 0.8978
---------------------------------------------
Finished training
Accuracy = 99.6149%
Precision = 0.9558
Recall = 0.8603
F1 Score = 0.9055
---------------------------------------------
Finished training
Accuracy = 99.6241%
Precision = 0.9734
Recall = 0.8432
F1 Score = 0.9036
---------------------------------------------


In [133]:
# Logistic Regression Classifier
for train_index, test_index in k_fold.split(scaled_df, labels):
    X_train, X_test = scaled_df[train_index], scaled_df[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    clf = logistic_regression.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    
    '''accuracy = accuracy_score(y_test, predictions) * 100

    metrics = precision_recall_fscore_support(y_test, predictions, average='macro')
    precision, recall = metrics[0], metrics[1]
    f1_score = 2 * (precision * recall) / (precision + recall)

    
    print("Accuracy = {0:.4f}%".format(accuracy))
    print("Precision = {0:.4f}".format(precision))
    print("Recall = {0:.4f}".format(recall))
    print("F1 Score = {0:.4f}".format(f1_score))'''
    print(classification_report(y_test, predictions))

    print("---------------------------------------------")
    
    

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    117489
           1       0.87      0.60      0.71      1440

    accuracy                           0.99    118929
   macro avg       0.93      0.80      0.85    118929
weighted avg       0.99      0.99      0.99    118929

---------------------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    117489
           1       0.86      0.58      0.69      1440

    accuracy                           0.99    118929
   macro avg       0.93      0.79      0.84    118929
weighted avg       0.99      0.99      0.99    118929

---------------------------------------------
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    117489
           1       0.87      0.58      0.70      1440

    accuracy                           0.99    118929
   macro avg       0.93      0.79   

## Extracting network features

In [23]:
# Connecting to the Neo4j database
graph = Graph(password="root")

# Query to fetch the network features from Neo4j
query = """
MATCH (p:Placeholder)
RETURN p.id AS id, p.degree AS degree, p.pagerank as pagerank, p.community AS community 
"""

data = graph.run(query)

records = {}

for record in data:
    records[record['id']] = {'degree': record['degree'], 'pagerank': record['pagerank'], 'community': record['community']}


In [24]:
banksim_df = pd.read_csv("../data/bs140513_032310.csv")


In [25]:
def load_degree(record):
    return records[record.split("'")[1]]['degree']
def load_community(record):
    return str(records[record.split("'")[1]]['community'])
def load_pagerank(record):
    return records[record.split("'")[1]]['pagerank']

In [26]:
banksim_df['merchant_degree'] = banksim_df['merchant'].apply(load_degree)
banksim_df['customer_degree'] = banksim_df['customer'].apply(load_degree)
banksim_df['merchant_pagerank'] = banksim_df['merchant'].apply(load_pagerank)
banksim_df['customer_pagerank'] = banksim_df['customer'].apply(load_pagerank)
banksim_df['merchant_community'] = banksim_df['merchant'].apply(load_community)
banksim_df['customer_community'] = banksim_df['customer'].apply(load_community)

In [27]:
banksim_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud,merchant_degree,customer_degree,customer_pagerank,merchant_pagerank,merchant_community,customer_community
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0,3929,7,0.15,46.541354,598811,598811
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0,3929,12,0.15,46.541354,598811,598846
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0,3573,14,0.15,41.391096,598846,598811
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0,3929,11,0.15,46.541354,598811,598846
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0,3929,14,0.15,46.541354,598811,598811
